In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import torch
print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("Device:", torch.device("cuda" if torch.cuda.is_available() else "cpu"))


# Check for CUDA
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Torch version: 2.5.1+cu121
CUDA available: True
Device: cuda
Using device: cuda


**Data Cleaning & Modeling Pipeline Plan**

1. Load and inspect data

2. Handle missing values

3. Feature engineering

4. Encode target

5. Train/test split

6. Baseline model

7. Evaluate and iterate



In [2]:
# Step 1: Load and inspect the data
import pandas as pd

# Load the data
train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")

if 'Personality' in test.columns:
    test = test.drop(columns=['Personality'])

# Basic overview
print(f"Train shape: {train.shape}")
print(f"Test shape: {test.shape}")
print("\nTrain info:")
train.info()

# Missing values
print("\nMissing values in train:")
print(train.isna().sum()[train.isna().sum() > 0].sort_values(ascending=False))

# Preview target
print("\nTarget value counts:")
print(train['Personality'].value_counts(dropna=False))



Train shape: (18524, 9)
Test shape: (6175, 8)

Train info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18524 entries, 0 to 18523
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         18524 non-null  int64  
 1   Time_spent_Alone           17334 non-null  float64
 2   Stage_fear                 16631 non-null  object 
 3   Social_event_attendance    17344 non-null  float64
 4   Going_outside              17058 non-null  float64
 5   Drained_after_socializing  17375 non-null  object 
 6   Friends_circle_size        17470 non-null  float64
 7   Post_frequency             17260 non-null  float64
 8   Personality                18524 non-null  object 
dtypes: float64(5), int64(1), object(3)
memory usage: 1.3+ MB

Missing values in train:
Stage_fear                   1893
Going_outside                1466
Post_frequency               1264
Time_spent_Alone           

**Dataset Summary**
**Shapes:**

    train: 18,524 rows, 9 columns

    test: 6,175 rows, 8 columns (no target)

|Column|Missing|Type|
|--|--|--|
|`Stage_fear`|1,893|object|
|`Going_outside`|1,466|float|
|`Post_frequency`|1,264|float|
|`Time_spent_Alone`|1,190|float|
|`Social_event_attendance`|1,180|float|
|`Drained_after_socializing`|1,149|object|
|`Friends_circle_size`|1,054|float|

**Target distribution:**

 -   Extrovert: 13,699 (~74%)

 -   Introvert: 4,825 (~26%)

 -  **Imbalanced target**, something I'll need to handle during training

**Next Steps** (**Step 2** Plan: Clean the Data)

I'll handle missing values carefully based on our EDA findings:

1. Numerical Columns (float):

    - Impute using correlated features, KNN Imputer.

    - Use linear correlation-based fill when there's a strong relationship

    - These include: Time_spent_Alone, Social_event_attendance, Going_outside, Friends_circle_size, Post_frequency

2. Categorical Columns (object):

    - For Stage_fear and Drained_after_socializing, I observed they correlate with missingness in numeric fields

    - So I can fill them using related categorical/numeric values (like Going_outside, Post_frequency) grouped mode

3. Outlier handling (optional but worth flagging for later — I might revisit this during model tuning).

4. Encode categorical variables:

    - Stage_fear, Drained_after_socializing, and Personality (target)

5. Create a was_missing_* binary flag for imputed values

    - Always a good idea as it gives models a shot at capturing patterns related to why data was missing.

6. Save cleaned dataset for training reuse.


In [3]:
# Add binary flags for each column with missing data
missing_cols = [
    'Time_spent_Alone',
    'Stage_fear',
    'Social_event_attendance',
    'Going_outside',
    'Drained_after_socializing',
    'Friends_circle_size',
    'Post_frequency'
]

for col in missing_cols:
    train[f'{col}_missing'] = train[col].isna().astype(int)
    test[f'{col}_missing'] = test[col].isna().astype(int)


**Step 3: Fill Missing Values**

- Numeric Columns -> KNN Imputer

- Categorical Columns ->   


**Define Groups of Correlated Features**

In [4]:

# Logical groupings based on correlation structure
group_social_behavior = ['Time_spent_Alone', 'Going_outside', 'Drained_after_socializing', 'Stage_fear']
group_social_networking = ['Friends_circle_size', 'Post_frequency']


**KNN Imputer for Numerical Columns**

In [5]:
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler


# Combine train and test for joint imputation
combined = pd.concat([train, test], keys=['train', 'test'])

# Numeric columns to impute and scale
knn_impute_cols = ['Time_spent_Alone', 'Social_event_attendance', 'Going_outside',
                   'Friends_circle_size', 'Post_frequency']

# Impute missing values
knn_imputer = KNNImputer(n_neighbors=5)
combined[knn_impute_cols] = knn_imputer.fit_transform(combined[knn_impute_cols])

# Split combined back into train and test
train = combined.xs('train').copy()
test = combined.xs('test').copy()

# Scale numeric columns
scaler = StandardScaler()
train.loc[:, knn_impute_cols] = scaler.fit_transform(train[knn_impute_cols])
test.loc[:, knn_impute_cols] = scaler.transform(test[knn_impute_cols])

train[knn_impute_cols].describe().T


,count,mean,std,min,25%,50%,75%,max
Time_spent_Alone,18524.0,7.211295e-17,1.000027,-1.050602,-0.714208,-0.377813,0.294975,2.649735
Social_event_attendance,18524.0,3.028360e-16,1.000027,-1.905702,-0.810873,-0.080987,0.648899,1.743728
Going_outside,18524.0,4.345956e-16,1.000027,-1.956687,-0.492413,-0.004321,0.971861,1.459953
Friends_circle_size,18524.0,1.077859e-16,1.000027,-1.900660,-0.707331,0.008666,0.724663,1.679326
Post_frequency,18524.0,4.660491e-17,1.000027,-1.720321,-0.672331,0.026330,0.724990,1.772981


**Predict Categorical Columns (Binary Classification)**

But first, before we forget again.

In [6]:
# Clean string-based binary columns while keeping NaNs intact
binary_map = {'Yes': 1, 'No': 0}
for col in ['Stage_fear', 'Drained_after_socializing']:
    train.loc[:, col] = train[col].apply(lambda x: binary_map[x] if x in binary_map else np.nan)
    test.loc[:, col] = test[col].apply(lambda x: binary_map[x] if x in binary_map else np.nan)


I'll predict missing values in `Stage_fear` and `Drained_after_socializing` by using logistic regression trained only on the rows with complete values:

In [7]:
from sklearn.linear_model import LogisticRegression

def impute_categorical(train_df, test_df, column, predictors, verbose=False):
    # Only use rows with known target
    known = train_df[train_df[column].notnull()]
    unknown = train_df[train_df[column].isnull()]

    if verbose:
        print(f"\n--- Imputing {column} ---")
        print("Label distribution:\n", known[column].value_counts())
        print("Predictors nulls:\n", known[predictors].isnull().sum())

    model = LogisticRegression(max_iter=1000)
    model.fit(known[predictors], known[column].astype(int))

    # Predict and fill missing values in train
    if not unknown.empty:
        train_df.loc[train_df[column].isnull(), column] = model.predict(unknown[predictors])

    # Predict and fill missing values in test
    test_unknown = test_df[test_df[column].isnull()]
    if not test_unknown.empty:
        test_df.loc[test_df[column].isnull(), column] = model.predict(test_unknown[predictors])


# Define predictor sets based on correlation analysis
predictors_stage_fear = ['Time_spent_Alone', 'Going_outside', 'Social_event_attendance']
predictors_drained = ['Time_spent_Alone', 'Stage_fear', 'Going_outside']

impute_categorical(train, test, 'Stage_fear', predictors_stage_fear, verbose=True)
impute_categorical(train, test, 'Drained_after_socializing', predictors_drained, verbose=True)

# Ensure binary categorical columns are cast to int after model prediction (they may be float)
train.loc[:, 'Stage_fear'] = train['Stage_fear'].astype(int)
test.loc[:, 'Stage_fear'] = test['Stage_fear'].astype(int)

train.loc[:, 'Drained_after_socializing'] = train['Drained_after_socializing'].astype(int)
test.loc[:, 'Drained_after_socializing'] = test['Drained_after_socializing'].astype(int)




--- Imputing Stage_fear ---
Label distribution:
 Stage_fear
0.0    12609
1.0     4022
Name: count, dtype: int64
Predictors nulls:
 Time_spent_Alone           0
Going_outside              0
Social_event_attendance    0
dtype: int64

--- Imputing Drained_after_socializing ---
Label distribution:
 Drained_after_socializing
0.0    13313
1.0     4062
Name: count, dtype: int64
Predictors nulls:
 Time_spent_Alone    0
Stage_fear          0
Going_outside       0
dtype: int64


**Encoding Categorical Variables**

In [8]:
# Safe encoding of target and binary columns with .loc to avoid SettingWithCopyWarning
train.loc[:, 'Personality'] = train['Personality'].map({'Introvert': 0, 'Extrovert': 1})
test.loc[:, 'Personality'] = test['Personality'].map({'Introvert': 0, 'Extrovert': 1})  # Only if used in eval

# Drop rows with missing target in training set
train = train[train['Personality'].notnull()].copy()
train['Personality'] = train['Personality'].astype(int)

binary_cols = ['Stage_fear', 'Drained_after_socializing']
for col in binary_cols:
    train.loc[:, col] = train[col].astype(int)
    test.loc[:, col] = test[col].astype(int)

# Show columns and data types
train.info()


<class 'pandas.core.frame.DataFrame'>
Index: 18524 entries, 0 to 18523
Data columns (total 16 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   id                                 18524 non-null  int64  
 1   Time_spent_Alone                   18524 non-null  float64
 2   Stage_fear                         18524 non-null  object 
 3   Social_event_attendance            18524 non-null  float64
 4   Going_outside                      18524 non-null  float64
 5   Drained_after_socializing          18524 non-null  object 
 6   Friends_circle_size                18524 non-null  float64
 7   Post_frequency                     18524 non-null  float64
 8   Personality                        18524 non-null  int64  
 9   Time_spent_Alone_missing           18524 non-null  int64  
 10  Stage_fear_missing                 18524 non-null  int64  
 11  Social_event_attendance_missing    18524 non-null  int64  


Let's take a moment to validate our janitorial efforts.

In [9]:
# 1. Confirm no missing values
print("Missing values in train:\n", train.isna().sum())
print("\nMissing values in test:\n", test.isna().sum())

# 2. Check dtypes and unique values of categorical (now numeric) columns
print("\nStage_fear unique:", train['Stage_fear'].unique())
print("Drained_after_socializing unique:", train['Drained_after_socializing'].unique())
print("Personality unique:", train['Personality'].unique())

# 3. Check target distribution
print("\nTarget value counts:")
print(train['Personality'].value_counts())

# 4. Sample preview
print("\nTrain sample:")
print(train.head())

# Show columns and data types
train.info()


Missing values in train:
 id                                   0
Time_spent_Alone                     0
Stage_fear                           0
Social_event_attendance              0
Going_outside                        0
Drained_after_socializing            0
Friends_circle_size                  0
Post_frequency                       0
Personality                          0
Time_spent_Alone_missing             0
Stage_fear_missing                   0
Social_event_attendance_missing      0
Going_outside_missing                0
Drained_after_socializing_missing    0
Friends_circle_size_missing          0
Post_frequency_missing               0
dtype: int64

Missing values in test:
 id                                      0
Time_spent_Alone                        0
Stage_fear                              0
Social_event_attendance                 0
Going_outside                           0
Drained_after_socializing               0
Friends_circle_size                     0
Post_frequency   

- spotless

---

Fix label formatting:

In [10]:
train = train.copy()
test = test.copy()

# Convert to int
train['Drained_after_socializing'] = train['Drained_after_socializing'].astype(int)
test['Drained_after_socializing'] = test['Drained_after_socializing'].astype(int)

train['Stage_fear'] = train['Stage_fear'].astype(int)
test['Stage_fear'] = test['Stage_fear'].astype(int)

# Show columns and data types
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18524 entries, 0 to 18523
Data columns (total 16 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   id                                 18524 non-null  int64  
 1   Time_spent_Alone                   18524 non-null  float64
 2   Stage_fear                         18524 non-null  int64  
 3   Social_event_attendance            18524 non-null  float64
 4   Going_outside                      18524 non-null  float64
 5   Drained_after_socializing          18524 non-null  int64  
 6   Friends_circle_size                18524 non-null  float64
 7   Post_frequency                     18524 non-null  float64
 8   Personality                        18524 non-null  int64  
 9   Time_spent_Alone_missing           18524 non-null  int64  
 10  Stage_fear_missing                 18524 non-null  int64  
 11  Social_event_attendance_missing    18524 non-null  int64  


In [11]:
# Save the cleaned datasets
train.to_csv("../data/cleaned_train.csv", index=False)
test.to_csv("../data/cleaned_test.csv", index=False)


---

Split the Training Set

- `X_train`, `X_valid`

- `y_train`, `y_valid`

For model evaluation before final test predictions.

Prepare training features and target by separating predictors from the target variable.

In [12]:
train = train[train['Personality'].notnull()].copy()
train['Personality'] = train['Personality'].astype(int)


X = train.drop(columns=['Personality', 'id'])  # keep ID only for post-pred join
y = train['Personality']

In [13]:
num_cols = [
    'Time_spent_Alone', 'Social_event_attendance', 'Going_outside',
    'Friends_circle_size', 'Post_frequency'
]

Standardize numeric feature columns to have zero mean and unit variance.

In [14]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = X.copy()
X_scaled[num_cols] = scaler.fit_transform(X[num_cols])

Create interaction features to capture relationships between social behavior and binary traits.

In [15]:
X_scaled['Alone_x_Fear'] = X_scaled['Time_spent_Alone'] * X_scaled['Stage_fear']
X_scaled['Social_x_Drained'] = X_scaled['Social_event_attendance'] * X_scaled['Drained_after_socializing']

Split the data into training and validation sets using stratified sampling to preserve class distribution.

In [16]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
import numpy as np

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

auc_scores = []

for fold, (train_idx, val_idx) in enumerate(skf.split(X_scaled, y)):
    X_train, X_val = X_scaled.iloc[train_idx], X_scaled.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
    
    clf = RandomForestClassifier(n_estimators=100, random_state=42)
    clf.fit(X_train, y_train)
    
    y_pred = clf.predict(X_val)
    y_proba = clf.predict_proba(X_val)[:, 1]
    
    auc = roc_auc_score(y_val, y_proba)
    auc_scores.append(auc)
    
    print(f"\nFold {fold+1} classification report:")
    print(classification_report(y_val, y_pred))
    print(f"AUC: {auc:.4f}")

print(f"\nMean AUC over folds: {np.mean(auc_scores):.4f}")




Fold 1 classification report:
              precision    recall  f1-score   support

           0       0.94      0.94      0.94       965
           1       0.98      0.98      0.98      2740

    accuracy                           0.97      3705
   macro avg       0.96      0.96      0.96      3705
weighted avg       0.97      0.97      0.97      3705

AUC: 0.9619

Fold 2 classification report:
              precision    recall  f1-score   support

           0       0.95      0.93      0.94       965
           1       0.97      0.98      0.98      2740

    accuracy                           0.97      3705
   macro avg       0.96      0.95      0.96      3705
weighted avg       0.97      0.97      0.97      3705

AUC: 0.9655

Fold 3 classification report:
              precision    recall  f1-score   support

           0       0.94      0.93      0.93       965
           1       0.97      0.98      0.98      2740

    accuracy                           0.97      3705
   macro av

Holy Crap! 95%

In [17]:
# 1. Capture feature names from training (pre-interactions)
features = X.columns.tolist()  # Still 14 columns

# 2. Prepare test features (copy to avoid modifying original)
test_scaled = test[features].copy()

# 3. Scale ONLY the numeric columns using the fitted scaler
test_scaled[num_cols] = scaler.transform(test_scaled[num_cols])

# 4. Add the same interaction features as in training
test_scaled['Alone_x_Fear'] = test_scaled['Time_spent_Alone'] * test_scaled['Stage_fear']
test_scaled['Social_x_Drained'] = test_scaled['Social_event_attendance'] * test_scaled['Drained_after_socializing']

# Now test_scaled has 16 columns, matching X_scaled

# Train on full data
final_model = RandomForestClassifier(n_estimators=100, random_state=42)
final_model.fit(X_scaled, y)

# 5. Predict
test_preds = final_model.predict(test_scaled)

# 6. Map predictions back to labels if Kaggle expects strings (check competition page)
# Assuming it wants 0/1, but if strings: test_preds = np.where(test_preds == 0, 'Introvert', 'Extrovert')

# 7. Save submission file
submission = pd.DataFrame({'id': test['id'], 'Personality': test_preds})
# Map back from 0/1 to string labels
submission['Personality'] = submission['Personality'].map({0: 'Introvert', 1: 'Extrovert'})
submission.to_csv('../submissions/submission_rf_baseline.csv', index=False)

print("✅ Submission file saved as 'submission_rf_baseline.csv'")

✅ Submission file saved as 'submission_rf_baseline.csv'


In [18]:
print(submission.shape)                       # (6175, 2)
print(submission.columns.tolist())            # ['id', 'Personality']
print(submission['Personality'].dtype)        # int64
print(submission['Personality'].unique())     # array([0, 1])
assert not submission['Personality'].isna().any()


(6175, 2)
['id', 'Personality']
object
['Extrovert' 'Introvert']


In [ ]:
print("✅ Shape:", submission.shape)
print("✅ Columns:", submission.columns.tolist())
print("✅ Dtypes:\n", submission.dtypes)
print("✅ Unique values:", submission['Personality'].unique())
print("✅ Any NaNs?", submission.isna().any().any())
print("✅ Any duplicates?", submission.duplicated(subset='id').any())
print("✅ Any missing IDs?", submission['id'].isna().any())
print("✅ ID range preview:", submission['id'].min(), "to", submission['id'].max())
print("✅ Sorted by ID?", submission['id'].is_monotonic_increasing)


